In [1]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import nilearn.image, nilearn.masking, nilearn.plotting
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
data_dir = '/Volumes/clmnlab/GA/MVPA/LSS_pb02_MO_short_duration/data/'

In [3]:
def fast_masking(img, roi):
    img_data = img.get_data()
    roi_mask = roi.get_data().astype(bool)
    
    if img_data.shape[:3] != roi_mask.shape:
        raise ValueError('different shape while masking! img=%s and roi=%s' % (img_data.shape, roi_mask.shape))
        
    return img_data[roi_mask, :].T

In [4]:
def accuracy(train_X, train_y, test_X, test_y):
    lda.fit(train_X, train_y)
    return np.mean(lda.predict(test_X) == test_y)

def cross_validation(datasets, target):
    train_y = target + target
    test_y = target
    
    result = []
    
    for i in range(len(datasets)):
        train_X = np.concatenate([a for j, a in enumerate(datasets) if j != i])
        test_X = datasets[i]
        
        result.append(accuracy(train_X, train_y, test_X, test_y))
        
    return np.mean(result)

In [5]:
roi_dir = '/Volumes/clmnlab/GA/fmri_data/masks/DMN/'

roi_path = [roi_dir + 'Average.nii']
#roi_path = sorted(glob.glob(roi_dir + '*.nii'))

roi_imgs = []

for path in roi_path:
    roi_imgs.append(nilearn.image.load_img(path))

In [6]:
roi_path

['/Volumes/clmnlab/GA/fmri_data/masks/DMN/Average.nii']

In [8]:
subj_list = [
    'GA01', 'GA02', 'GA05', 'GA07', 'GA08', 'GA11', 'GA12', 'GA13', 'GA14', 'GA15',
    'GA18', 'GA19', 'GA20', 'GA21', 'GA23', 'GA26', 'GA27', 'GA28', 'GA29', 'GA30',
    'GA31', 'GA32', 'GA33', 'GA34', 'GA35', 'GA36', 'GA37', 'GA38', 'GA42', 'GA44'
]
    
#    'GB01', 'GB02', 'GB05', 'GB07', 'GB08', 'GB11', 'GB12', 'GB13', 'GB14', 'GB15',
#    'GB18', 'GB19', 'GB20', 'GB21', 'GB23', 'GB26', 'GB27', 'GB28', 'GB29', 'GB30',
#    'GB31', 'GB32', 'GB33', 'GB34', 'GB35', 'GB36', 'GB37', 'GB38', 'GB42', 'GB44'
#]

In [9]:
data = {}

for subj in subj_list:
    for run in range(1, 7):
        data[subj, run] = nilearn.image.load_img(data_dir + 'betasLSS.MO.shortdur.%s.r%02d.nii.gz' % (subj, run))
    print(subj, end='\r')

In [10]:
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

In [11]:
target_path = list(range(1, 13)) * 8
target_pos = []

with open('/Volumes/clmnlab/GA/MVPA/LSS_pb02/behaviors/targetID.txt') as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]

In [12]:
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

results = {}
 
for subj in subj_list:
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        
        if ('pos', 'unpracticed', subj, roi_index) in results:
            continue
        
        X = [
            fast_masking(roi=roi, img=data[subj, 1]), 
            fast_masking(roi=roi, img=data[subj, 2]), 
            fast_masking(roi=roi, img=data[subj, 3]), 
        ]
        
        results['pos', 'practiced', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'practiced', subj, roi_index] = cross_validation(X, target_path)
        
        X = [
            fast_masking(roi=roi, img=data[subj, 4]), 
            fast_masking(roi=roi, img=data[subj, 5]), 
            fast_masking(roi=roi, img=data[subj, 6]), 
        ]
        
        results['pos', 'unpracticed', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'unpracticed', subj, roi_index] = cross_validation(X, target_path)
        
        print(subj, roi_index, end='\r')

In [13]:
len(results)

120

In [14]:
results

{('pos', 'practiced', 'GA01', 1): 0.40277777777777773,
 ('path', 'practiced', 'GA01', 1): 0.18402777777777776,
 ('pos', 'unpracticed', 'GA01', 1): 0.3506944444444444,
 ('path', 'unpracticed', 'GA01', 1): 0.18055555555555555,
 ('pos', 'practiced', 'GA02', 1): 0.3506944444444444,
 ('path', 'practiced', 'GA02', 1): 0.12499999999999999,
 ('pos', 'unpracticed', 'GA02', 1): 0.3854166666666667,
 ('path', 'unpracticed', 'GA02', 1): 0.14583333333333334,
 ('pos', 'practiced', 'GA05', 1): 0.3333333333333333,
 ('path', 'practiced', 'GA05', 1): 0.12499999999999999,
 ('pos', 'unpracticed', 'GA05', 1): 0.44097222222222215,
 ('path', 'unpracticed', 'GA05', 1): 0.24305555555555558,
 ('pos', 'practiced', 'GA07', 1): 0.34375,
 ('path', 'practiced', 'GA07', 1): 0.16666666666666666,
 ('pos', 'unpracticed', 'GA07', 1): 0.30902777777777785,
 ('path', 'unpracticed', 'GA07', 1): 0.17361111111111113,
 ('pos', 'practiced', 'GA08', 1): 0.2916666666666667,
 ('path', 'practiced', 'GA08', 1): 0.11458333333333333,
 (

In [15]:
data = []

In [16]:
subj_list = [
#    'GA01', 'GA02', 'GA05', 'GA07', 'GA08', 'GA11', 'GA12', 'GA13', 'GA14', 'GA15',
#    'GA18', 'GA19', 'GA20', 'GA21', 'GA23', 'GA26', 'GA27', 'GA28', 'GA29', 'GA30',
#    'GA31', 'GA32', 'GA33', 'GA34', 'GA35', 'GA36', 'GA37', 'GA38', 'GA42', 'GA44'
#]
    
   'GB01', 'GB02', 'GB05', 'GB07', 'GB08', 'GB11', 'GB12', 'GB13', 'GB14', 'GB15',
   'GB18', 'GB19', 'GB20', 'GB21', 'GB23', 'GB26', 'GB27', 'GB28', 'GB29', 'GB30',
   'GB31', 'GB32', 'GB33', 'GB34', 'GB35', 'GB36', 'GB37', 'GB38', 'GB42', 'GB44'
]

data = {}

for subj in subj_list:
    for run in range(1, 7):
        data[subj, run] = nilearn.image.load_img(data_dir + 'betasLSS.MO.shortdur.%s.r%02d.nii.gz' % (subj, run))
    print(subj, end='\r')

In [17]:
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

In [18]:
target_path = list(range(1, 13)) * 8
target_pos = []

with open('/Volumes/clmnlab/GA/MVPA/LSS_pb02/behaviors/targetID.txt') as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]

In [19]:
for subj in subj_list:
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        
        if ('pos', 'unpracticed', subj, roi_index) in results:
            continue
        
        X = [
            fast_masking(roi=roi, img=data[subj, 1]), 
            fast_masking(roi=roi, img=data[subj, 2]), 
            fast_masking(roi=roi, img=data[subj, 3]), 
        ]
        
        results['pos', 'practiced', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'practiced', subj, roi_index] = cross_validation(X, target_path)
        
        X = [
            fast_masking(roi=roi, img=data[subj, 4]), 
            fast_masking(roi=roi, img=data[subj, 5]), 
            fast_masking(roi=roi, img=data[subj, 6]), 
        ]
        
        results['pos', 'unpracticed', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'unpracticed', subj, roi_index] = cross_validation(X, target_path)
        
        print(subj, roi_index, end='\r')

In [20]:
df = pd.DataFrame(columns=['target', 'practiced', 'subject', 'roi', 'accu'])
for i,j in zip(results.keys(), results.values()):
    df = df.append({'target':i[0], 'practiced':i[1], 'subject':i[2], 'roi':i[3], 'accu':j}, ignore_index=True)
    
df.head()

,target,practiced,subject,roi,accu
0,pos,practiced,GA01,1,0.402778
1,path,practiced,GA01,1,0.184028
2,pos,unpracticed,GA01,1,0.350694
3,path,unpracticed,GA01,1,0.180556
4,pos,practiced,GA02,1,0.350694


In [23]:
df.to_csv("20190906_DMN_lda_average.csv")